In [5]:
import pandas as pd
import numpy as np

In [6]:
path = 'Resources/Amazon_rev.csv'

In [10]:
amazon_foods_df = pd.read_csv(path)
amazon_foods_df = amazon_foods_df.drop_column('Unnamed')

AttributeError: 'DataFrame' object has no attribute 'drop_column'

In [ ]:
amazon_foods_df.head()

In [9]:
len(amazon_foods_df)

525814

In [ ]:
amazon_foods_modified_df = amazon_foods_df